In [1]:
# ===================================================================
# 1. 라이브러리 임포트 (Import Libraries)
# ===================================================================
import numpy as np
import h5py
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Flatten, Dense, Dropout
import os
import warnings

# 불필요한 경고 메시지 숨기기
warnings.filterwarnings('ignore', category=FutureWarning)
tf.get_logger().setLevel('ERROR')
print("라이브러리 임포트 완료!")


# ===================================================================
# 2. 데이터 로드 및 전처리 (Load and Preprocess Data)
# ===================================================================
# ❗ 파일 경로를 본인의 환경에 맞게 수정하세요.
file_path = '/content/drive/MyDrive/p044036.mat'
X_signals, y_bp = None, None # 변수 초기화

try:
    with h5py.File(file_path, 'r') as mat_data:
        print(f"'{file_path}' 파일 로딩 성공 (HDF5 형식)!")

        TARGET_LENGTH = 1250

        def extract_signal_data(refs_dataset):
            """신호 데이터를 추출하고 길이를 통일하는 함수"""
            segments = []
            for ref in refs_dataset[0, :]:
                segment_data = mat_data[ref][:].flatten()
                if len(segment_data) > TARGET_LENGTH:
                    segment_data = segment_data[:TARGET_LENGTH]
                elif len(segment_data) < TARGET_LENGTH:
                    padding = np.zeros(TARGET_LENGTH - len(segment_data))
                    segment_data = np.concatenate([segment_data, padding])
                segments.append(segment_data)
            return np.array(segments, dtype=np.float32)

        def extract_label_data(refs_dataset):
            """레이블 데이터를 추출하는 함수"""
            labels = []
            for ref in refs_dataset[0, :]:
                value = mat_data[ref][0, 0]
                labels.append(value)
            return np.array(labels, dtype=np.float32)

        # --- ❗ 수정된 부분 1: PPG 신호만 추출 ---
        ppg_segments = extract_signal_data(mat_data['Subj_Wins']['PPG_F'])
        # ecg_segments = extract_signal_data(mat_data['Subj_Wins']['ECG_F']) # ECG 부분 제거

        sbp_labels = extract_label_data(mat_data['Subj_Wins']['SegSBP'])
        dbp_labels = extract_label_data(mat_data['Subj_Wins']['SegDBP'])

    print(f"\n데이터 추출 및 변환 최종 성공!")

    num_segments_to_use = 1600
    if len(ppg_segments) < num_segments_to_use:
        print(f"경고: 전체 세그먼트 개수({len(ppg_segments)})가 1600개보다 작아, 사용 가능한 {len(ppg_segments)}개만 사용합니다.")
        num_segments_to_use = len(ppg_segments)

    ppg_segments = ppg_segments[:num_segments_to_use]
    # ecg_segments = ecg_segments[:num_segments_to_use] # ECG 부분 제거
    sbp_labels = sbp_labels[:num_segments_to_use]
    dbp_labels = dbp_labels[:num_segments_to_use]

    # --- ❗ 수정된 부분 2: PPG 신호만으로 입력 데이터 구성 ---
    # np.expand_dims를 사용해 채널 차원(1)을 추가해줍니다.
    X_signals = np.expand_dims(ppg_segments, axis=-1)
    y_bp = np.stack([sbp_labels, dbp_labels], axis=-1)

    print(f"전처리 후 입력 데이터 X shape: {X_signals.shape}, dtype: {X_signals.dtype}")
    print(f"전처리 후 타겟 데이터 y shape: {y_bp.shape}, dtype: {y_bp.dtype}")

except FileNotFoundError:
    print(f"\n오류: '{file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"\n데이터 처리 중 얘기치 못한 오류가 발생했습니다: {e}")


# ===================================================================
# 3. K-Fold 교차 검증 및 모델 학습/평가 (이후 코드는 동일)
# ===================================================================
if X_signals is not None and y_bp is not None:
    N_SPLITS_REQUESTED = 4
    SEGMENTS_PER_FOLD = 400
    TRAIN_SIZE = 300
    TEST_SIZE = 100
    TOTAL_SEGMENTS = X_signals.shape[0]

    N_SPLITS = TOTAL_SEGMENTS // SEGMENTS_PER_FOLD
    if N_SPLITS < N_SPLITS_REQUESTED:
        print(f"\n경고: 데이터가 부족하여({TOTAL_SEGMENTS}개) 요청된 4-Fold가 아닌 {N_SPLITS}-Fold로 교차 검증을 실행합니다.")

    if N_SPLITS == 0:
        print("\n오류: 데이터가 1개 Fold(400 세그먼트)를 구성하기에 부족하여 학습을 중단합니다.")
    else:
        sbp_maes, dbp_maes = [], []

        for k in range(N_SPLITS):
            print("\n" + "="*50)
            print(f" FOLD {k+1}/{N_SPLITS} 시작 ".center(50, "="))
            print("="*50)

            fold_start_idx = k * SEGMENTS_PER_FOLD
            train_indices = range(fold_start_idx, fold_start_idx + TRAIN_SIZE)
            test_indices = range(fold_start_idx + TRAIN_SIZE, fold_start_idx + TRAIN_SIZE + TEST_SIZE)

            X_train, y_train = X_signals[train_indices], y_bp[train_indices]
            X_test, y_test = X_signals[test_indices], y_bp[test_indices]

            print(f"Fold {k+1}: 훈련 데이터 {len(X_train)}개, 테스트 데이터 {len(X_test)}개")

            scaler = StandardScaler()
            n_samples_train, n_timesteps, n_features = X_train.shape

            X_train_reshaped = X_train.reshape(-1, n_features)
            scaler.fit(X_train_reshaped)
            X_train_scaled = scaler.transform(X_train_reshaped).reshape(n_samples_train, n_timesteps, n_features)

            if X_test.shape[0] > 0:
                n_samples_test = X_test.shape[0]
                X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(n_samples_test, n_timesteps, n_features)
            else:
                X_test_scaled = X_test

            print("데이터 스케일링 완료.")

            input_shape = (X_signals.shape[1], X_signals.shape[2])
            model = Sequential([
                Input(shape=input_shape),
                Conv1D(filters=64, kernel_size=7, activation='relu', padding='same'),
                MaxPooling1D(pool_size=2),
                Conv1D(filters=128, kernel_size=5, activation='relu', padding='same'),
                MaxPooling1D(pool_size=2),
                LSTM(units=100, return_sequences=False),
                Dropout(0.3),
                Dense(128, activation='relu'),
                Dropout(0.3),
                Dense(64, activation='relu'),
                Dense(2, activation='linear')
            ])
            model.compile(optimizer='adam', loss='mean_absolute_error')

            if k == 0:
                print("\n--- 모델 구조 ---")
                model.summary()

            print("\n모델 학습 시작...")
            history = model.fit(
                X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)]
            )
            print(f"Fold {k+1}: 모델 학습 완료 (Best epoch at: {np.argmin(history.history['val_loss']) + 1})")

            predictions = model.predict(X_test_scaled, verbose=0)
            fold_sbp_mae = mean_absolute_error(y_test[:, 0], predictions[:, 0])
            fold_dbp_mae = mean_absolute_error(y_test[:, 1], predictions[:, 1])
            sbp_maes.append(fold_sbp_mae)
            dbp_maes.append(fold_dbp_mae)

            print(f"\n--- FOLD {k+1} 결과 ---")
            print(f"SBP MAE: {fold_sbp_mae:.4f}")
            print(f"DBP MAE: {fold_dbp_mae:.4f}")

        # ===================================================================
        # 4. 최종 결과 출력 (Final Results)
        # ===================================================================
        if sbp_maes and dbp_maes:
            print("\n" + "="*50)
            print(" 최종 교차 검증 결과 ".center(50, "="))
            print("="*50)
            print(f"SBP 최종 MAE: {np.mean(sbp_maes):.4f} ± {np.std(sbp_maes):.4f}")
            print(f"DBP 최종 MAE: {np.mean(dbp_maes):.4f} ± {np.std(dbp_maes):.4f}")
else:
    print("\n데이터셋이 정상적으로 로드되지 않아 학습을 진행할 수 없습니다.")

라이브러리 임포트 완료!
'/content/drive/MyDrive/p044036.mat' 파일 로딩 성공 (HDF5 형식)!

데이터 추출 및 변환 최종 성공!
전처리 후 입력 데이터 X shape: (1600, 1250, 1), dtype: float32
전처리 후 타겟 데이터 y shape: (1600, 2), dtype: float32

================== FOLD 1/4 시작 ===================
Fold 1: 훈련 데이터 300개, 테스트 데이터 100개
데이터 스케일링 완료.

--- 모델 구조 ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 1250, 64)       │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 625, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 625, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 312, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 154,514 (603.57 KB)

 Trainable params: 154,514 (603.57 KB)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Fold 1: 모델 학습 완료 (Best epoch at: 12)

--- FOLD 1 결과 ---
SBP MAE: 4.6749
DBP MAE: 2.7585

================== FOLD 2/4 시작 ===================
Fold 2: 훈련 데이터 300개, 테스트 데이터 100개
데이터 스케일링 완료.

모델 학습 시작...
Fold 2: 모델 학습 완료 (Best epoch at: 24)

--- FOLD 2 결과 ---
SBP MAE: 7.4460
DBP MAE: 3.9953

================== FOLD 3/4 시작 ===================
Fold 3: 훈련 데이터 300개, 테스트 데이터 100개
데이터 스케일링 완료.

모델 학습 시작...
Fold 3: 모델 학습 완료 (Best epoch at: 6)

--- FOLD 3 결과 ---
SBP MAE: 1.5549
DBP MAE: 1.9542

================== FOLD 4/4 시작 ===================
Fold 4: 훈련 데이터 300개, 테스트 데이터 100개
데이터 스케일링 완료.

모델 학습 시작...
Fold 4: 모델 학습 완료 (Best epoch at: 17)

--- FOLD 4 결과 ---
SBP MAE: 4.3264
DBP MAE: 2.4858

================== 최종 교차 검증 결과 ===================
SBP 최종 MAE: 4.5005 ± 2.0865
DBP 최종 MAE: 2.7985 ± 0.7491
